In [1]:
from logger import MyLogger
import SubjObjLoader
import json
from torch import nn
import torch
from pytorch_transformers import *
import importlib
from collections import deque
# import dataloader
from BertRDMLoader import *
import json
from torch import nn
import torch
from pytorch_transformers import *
import importlib
from tensorboardX import SummaryWriter
import torch.nn.utils.rnn as rnn_utils
import tsentiLoader

In [2]:
from emotionLoader import *
from SubjObjLoader import *
import numpy as np

### BERT RDM CM 模型代码
> 需要改动的地方
> - 损失函数要从训练函数中拆分出来，方便后面的联合训练
> 

In [3]:
class pooling_layer(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(pooling_layer, self).__init__()
        self.linear = nn.Linear(input_dim, output_dim)
        self.input_dim = input_dim
        self.output_dim = output_dim
        
    def forward(self, inputs):
        assert(inputs.ndim == 4 ) # [batchsize, max_seq_len, max_word_num, input_dim] 
        batch_size, max_seq_len, max_word_num, input_dim = inputs.shape
        assert(input_dim == self.input_dim)
        t_inputs = inputs.reshape([-1, self.input_dim])
        return self.linear(t_inputs).reshape(
            
            [-1, max_word_num, self.output_dim]
        
        ).max(axis=1)[0].reshape(
        
            [-1, max_seq_len, self.output_dim]
        
        )

class RDM_Model(nn.Module):
    def __init__(self, word_embedding_dim, sent_embedding_dim, hidden_dim, dropout_prob):
        super(RDM_Model, self).__init__()
        self.embedding_dim = sent_embedding_dim
        self.hidden_dim = hidden_dim
        self.gru_model = nn.GRU(word_embedding_dim, 
                                self.hidden_dim, 
                                batch_first=True, 
                                dropout=dropout_prob
                            )
        self.DropLayer = nn.Dropout(dropout_prob)
#         self.PoolLayer = pooling_layer(word_embedding_dim, sent_embedding_dim) 
        
    def forward(self, x_emb, x_len, init_states): 
        """
        input_x: [batchsize, max_seq_len, sentence_embedding_dim] 
        x_emb: [batchsize, max_seq_len, 1, embedding_dim]
        x_len: [batchsize]
        init_states: [batchsize, hidden_dim]
        """
        batchsize, max_seq_len, _ , emb_dim = x_emb.shape
#         pool_feature = self.PoolLayer(x_emb)
#         sent_feature = sentiModel( 
#                 x_emb.reshape(
#                     [-1, max_sent_len, emb_dim]
#                 ) 
#             ).reshape(
#                 [batchsize, max_seq_len, -1]
#             )
#         pooled_input_x_dp = self.DropLayer(input_x)
        pool_feature = x_emb.reshape(
                [-1, max_seq_len, emb_dim]
        )
        df_outputs, df_last_state = self.gru_model(pool_feature, init_states)
        hidden_outs = [df_outputs[i][:x_len[i]] for i in range(batchsize)]
        final_outs = [df_outputs[i][x_len[i]-1] for i in range(batchsize)]
        return hidden_outs, final_outs


class CM_Model(nn.Module):
    def __init__(self, sentence_embedding_dim, hidden_dim, action_num):
        super(CM_Model, self).__init__()
        self.sentence_embedding_dim = sentence_embedding_dim
        self.hidden_dim = hidden_dim
        self.action_num = action_num
#         self.PoolLayer = pooling_layer(self.embedding_dim, 
#                                             self.hidden_dim)
        self.DenseLayer = nn.Linear(self.hidden_dim, 64)
        self.Classifier = nn.Linear(64, self.action_num)
        
    def forward(self, rdm_model, s_model, rl_input, rl_state):
        """
        rl_input: [batchsize, max_word_num, sentence_embedding_dim]
        rl_state: [1, batchsize, hidden_dim]
        """
        assert(rl_input.ndim==3)
        batchsize, max_word_num, embedding_dim = rl_input.shape
#         assert(embedding_dim==self.embedding_dim)
        sentence = s_model(rl_input).reshape(batch_size, 1, self.sentence_embedding_dim)
#         pooled_rl_input = self.PoolLayer(
#             rl_input.reshape(
#                 [-1, 1, max_word_num, self.embedding_dim]
#             )
#         ).reshape([-1, 1, self.hidden_dim])
        
#         print("sentence:", sentence.shape)
#         print("rl_state:", rl_state.shape)
        rl_output, rl_new_state = rdm_model.gru_model(
                                            sentence, 
                                            rl_state
                                        )
        rl_h1 = nn.functional.relu(
            self.DenseLayer(
#                 rl_state.reshape([len(rl_input), self.hidden_dim]) #it is not sure to take rl_state , rather than rl_output, as the feature
                rl_output.reshape(
                    [len(rl_input), self.hidden_dim]
                )
            )
        )
        stopScore = self.Classifier(rl_h1)
        isStop = stopScore.argmax(axis=1)
        return stopScore, isStop, rl_new_state


# In[3]:


def layer2seq(bert, layer, cuda=False):
    if cuda:
        outs = [bert( torch.tensor([input_]).cuda())
                for input_ in layer]   
    else: 
        outs = [bert( torch.tensor([input_]))
                    for input_ in layer]
    states = [item[1] for item in outs]
    return rnn_utils.pad_sequence(states, batch_first=True)

def Word_ids2SeqStates(word_ids, bert, ndim, cuda=False):
    assert(ndim == 3)
    if cuda:
        embedding = [layer2seq(bert, layer, cuda) for layer in word_ids]
    else:
        embedding = [layer2seq(bert, layer) for layer in word_ids]
    return padding_sequence(embedding)


# In[4]:


def Count_Accs(ylabel, preds):
    correct_preds = np.array(
        [1 if y1==y2 else 0 
        for (y1, y2) in zip(ylabel, preds)]
    )
    y_idxs = [idx if yl >0 else idx - len(ylabel) 
            for (idx, yl) in enumerate(ylabel)]
    pos_idxs = list(filter(lambda x: x >= 0, y_idxs))
    neg_idxs = list(filter(lambda x: x < 0, y_idxs))
    acc = sum(correct_preds) / (1.0 * len(ylabel))
    if len(pos_idxs) > 0:
        pos_acc = sum(correct_preds[pos_idxs])/(1.0*len(pos_idxs))
    else:
        pos_acc = 0
    if len(neg_idxs) > 0:
        neg_acc = sum(correct_preds[neg_idxs])/(1.0*len(neg_idxs))
    else:
        neg_acc = 0
    return acc, pos_acc, neg_acc, y_idxs, pos_idxs, neg_idxs, correct_preds

def Loss_Fn(ylabel, pred_scores):
    diff = ((ylabel - pred_scores)*(ylabel - pred_scores)).mean(axis=1)
#     pos_neg = (1.0*sum(ylabel.argmax(axis=1)))/(1.0*(len(ylabel) - sum(ylabel.argmax(axis=1))))
    pos_neg = 0
    if pos_neg > 0:
        print("unbalanced data")
        weight = torch.ones(len(ylabel)).cuda() + (ylabel.argmax(axis=1).to(torch.float32)/(1.0*pos_neg)) - ylabel.argmax(axis=1).to(torch.float32)
        return (weight *diff).mean()
    else:
        print("totally unbalanced data")
        return diff.mean()


# In[5]:
def rdm_loss(x, y, bert, rdm_model, rdm_classifier, loss_fn):
    x_emb = Word_ids2SeqStates(x, bert, 3, cuda=True) 
    batchsize, max_seq_len, max_sent_len, emb_dim = x_emb.shape
    rdm_hiddens, rdm_outs = rdm_model(x_emb, x_len, init_states)
    rdm_scores = rdm_classifier(
        torch.cat(
            rdm_outs # a list of tensor, where the ndim of tensor is 1 and the shape of tensor is [hidden_size]
        ).reshape(
            [-1, rdm_model.hidden_dim]
        )
    )
    rdm_preds = rdm_scores.argmax(axis=1)
    y_label = y.argmax(axis=1)
    acc, _, _, _, _, _, _ = Count_Accs(y_label, rdm_preds)
    loss = loss_fn(rdm_scores, torch.tensor(y_label).cuda())
    return loss, acc


def TrainRDMModel(rdm_model, bert, rdm_classifier, 
                    tokenizer, t_steps, new_data_len=[], logger=None, 
                        log_dir="RDMBertTrain"):
    batch_size = 20 
    max_gpu_batch = 5 #cannot load a larger batch into the limited memory, but we could  accumulates grads
    assert(batch_size%max_gpu_batch == 0)
    sum_loss = 0.0
    sum_acc = 0.0
    t_acc = 0.9
    ret_acc = 0.0
    init_states = torch.zeros([1, 5, rdm_model.hidden_dim], dtype=torch.float32).cuda()
    weight = torch.tensor([2.0, 1.0], dtype=torch.float32).cuda()
    loss_fn = nn.CrossEntropyLoss(weight=weight)
    optim = torch.optim.Adagrad([
                                {'params': bert.parameters(), 'lr':5e-5},
                                {'params': rdm_classifier.parameters(), 'lr': 5e-5},
                                {'params': rdm_model.parameters(), 'lr': 5e-5}
                             ]
    )
    
    writer = SummaryWriter(log_dir)
    acc_l = np.zeros(int(batch_size/max_gpu_batch))
    loss_l = np.zeros(int(batch_size/max_gpu_batch))
    for step in range(t_steps):
        optim.zero_grad()
        for j in range(int(batch_size/max_gpu_batch)):
            if len(new_data_len) > 0:
                x, x_len, y = get_df_batch(step, max_gpu_batch, new_data_len, tokenizer=tokenizer)
            else:
                x, x_len, y = get_df_batch(step, max_gpu_batch, tokenizer=tokenizer)
                
            loss, acc = rdm_loss(x, y, bert, rdm_model, rdm_classifier, loss_fn)
            loss.backward()
            loss_l[j] = loss
            acc_l[j] = acc
            
        optim.step()        
        writer.add_scalar('Train Loss', loss_l.mean(), step)
        writer.add_scalar('Train Accuracy', acc_l.mean(), step)

        sum_loss += loss_l.mean()
        sum_acc += acc_l.mean()
        

        if step % 10 == 9:
            sum_loss = sum_loss / 10
            sum_acc = sum_acc / 10
            print('%3d | %d , train_loss/accuracy = %6.8f/%6.7f'             % (step, t_steps, 
                sum_loss, sum_acc,
                ))
            logger.info('%3d | %d , train_loss/accuracy = %6.8f/%6.7f'             % (step, t_steps, 
                sum_loss, sum_acc,
                ))
            if step%500 == 499:
                rdm_save_as = '/home/hadoop/ERD/%s/rdmModel_epoch%03d.pkl'                                    % (log_dir, step/500, sum_acc)
                torch.save(
                    {
                        "bert":bert.state_dict(),
                        "sentiModel":sentiModel.state_dict(),
                        "rmdModel":rdm_model.state_dict(),
                        "rdm_classifier": rdm_classifierdm.state_dict()
                    },
                    rdm_save_as
                )
#                 rdm_model, bert, sentiModel, rdm_classifier
            if sum_acc > t_acc:
                break
            sum_acc = 0.0
            sum_loss = 0.0

    print(get_curtime() + " Train df Model End.")
    logger.info(get_curtime() + " Train df Model End.")
    return ret_acc


# In[6]:


def TrainCMModel(bert, rdm_model, rdm_classifier, cm_model, tokenizer, log_dir, logger, FLAGS):
    batch_size = 20
    t_acc = 0.9
    ids = np.array(range(batch_size), dtype=np.int32)
    seq_states = np.zeros([batch_size], dtype=np.int32)
    isStop = np.zeros([batch_size], dtype=np.int32)
    max_id = batch_size
    df_init_states = torch.zeros([1, batch_size, FLAGS.hidden_dim], dtype=torch.float32).cuda()
    state = df_init_states
    D = deque()
    ssq = []
    print("in RL the begining")
    rdm_optim = torch.optim.Adagrad([
                            {'params': bert.parameters(), 'lr':1e-3},
    #                                 {'params': rdm_classifier.parameters(), 'lr': 5e-2},
                            {'params': rdm_model.parameters(), 'lr': 5e-2},
                            {'params': sentiModel.parameters(), 'lr': 1e-2}
                         ],
                            weight_decay = 0.2
    )
    rl_optim = torch.optim.Adam([{'params':cm_model.parameters(), 'lr':1e-3}])
    # get_new_len(sess, mm)
    data_ID = get_data_ID()

    if len(data_ID) % batch_size == 0: # the total number of events
        flags = int(len(data_ID) / FLAGS.batch_size)
    else:
        flags = int(len(data_ID) / FLAGS.batch_size) + 1
    for i in range(flags):
        with torch.no_grad():
            x, x_len, y = get_df_batch(i, batch_size, tokenizer=tokenizer)
            x_emb = Word_ids2SeqStates(x, bert, 3, cuda=True) 
            batchsize, max_seq_len, max_sent_len,                                     emb_dim = x_emb.shape
            sent_feature = sentiModel( 
                x_emb.reshape(
                    [-1, max_sent_len, emb_dim]
                ) 
            ).reshape(
                [batchsize, max_seq_len, -1]
            )
            rdm_hiddens, rdm_outs = rdm_model(sent_feature, x_len, df_init_states)
        #         t_ssq = sess.run(rdm_train.out_seq, feed_dic)# t_ssq = [batchsize, max_seq, scores]
            print("batch %d"%i)
            if len(ssq) > 0:
                ssq.extend([rdm_classifier(h) for h in rdm_hiddens])
            else:
                ssq = [rdm_classifier(h) for h in rdm_hiddens]

    print(get_curtime() + " Now Start RL training ...")
    counter = 0
    sum_rw = 0.0 # sum of rewards

    data_len = get_data_len()

    while True:
        if counter > FLAGS.OBSERVE:
            sum_rw += np.mean(rw)
            if counter % 200 == 0:
                sum_rw = sum_rw / 2000
                print( get_curtime() + " Step: " + str(step) 
                       + " REWARD IS " + str(sum_rw) 
                     )
                logger.info( get_curtime() + 
                             " Step: " + str(step) + 
                            " REWARD IS " + str(sum_rw)
                           )
                if sum_rw > t_rw:
                    print("Retch The Target Reward")
                    logger.info("Retch The Target Reward")
                    break
                if counter > t_steps:
                    print("Retch The Target Steps")
                    logger.info("Retch The Target Steps")
                    break
                sum_rw = 0.0
            s_state, s_x, s_isStop, s_rw = get_RL_Train_batch(D, FLAGS)
            stopScore, isStop, rl_new_state = cm_model(rdm_model, sentiModel, s_x, s_state)
            out_action = (stopScore*s_isStop).sum(axis=1)
            rl_cost = torch.mean((s_rw - out_action)*(s_rw - out_action))
            rl_cost.backward()
            rl_optim.step()

        input_x, input_y, ids, seq_states, max_id = get_rl_batch(ids, seq_states, isStop, max_id, 0, FLAGS, tokenizer=tokenizer)
        with torch.no_grad():
            x_emb = layer2seq(bert, input_x, cuda=True)
            batchsize, max_sent_len, emb_dim = x_emb.shape
            mss, isStop, mNewState = cm_model(rdm_model, sentiModel, x_emb, state)

        for j in range(FLAGS.batch_size):
            if random.random() < FLAGS.random_rate:
    #             isStop[j] = np.argmax(np.random.rand(2))
                isStop[j] = int(torch.rand(2).argmax())
            if seq_states[j] == data_len[ids[j]]:
                isStop[j] = 1

        # eval
        rw = get_reward(isStop, mss, ssq, ids, seq_states)

        for j in range(FLAGS.batch_size):
            D.append((state[0][j], input_x[j], isStop[j], rw[j]))
            if len(D) > FLAGS.max_memory:
                D.popleft()

        state = mNewState
        for j in range(FLAGS.batch_size):
            if isStop[j] == 1:
                # init_states = np.zeros([FLAGS.batch_size, FLAGS.hidden_dim], dtype=np.float32)
                # feed_dic = {rl_model.init_states: init_states}
                # state[j] = sess.run(rl_model.df_state, feed_dic)
    #             state[j] = np.zeros([FLAGS.hidden_dim], dtype=np.float32)
                state[0][j].fill_(0)
        counter += 1.0

## 联合学习部分

### 读数据
 zip 三个数据集，一次性全部读出来



### 工具函数

In [4]:
def words2tensor(bert, layer, cuda=False):
    if cuda:
        outs = [bert( torch.tensor([input_]).cuda())
                for input_ in layer]   
    else: 
        outs = [bert( torch.tensor([input_]))
                    for input_ in layer]
    states = [item[0][0] for item in outs]
    return rnn_utils.pad_sequence(states, batch_first=True)


> normalized the data for unbalanced learning:
```python
weights = WeightsForUmbalanced(emoReader.label)
weights
```

In [5]:
def WeightsForUmbalanced(data_label):
    _, _, labels = data_label.shape
    label_cnt = data_label.reshape([-1, labels]).sum(axis=0)
    weights = 1.0/label_cnt
    normalized_weights = weights/sum(weights)
    return normalized_weights

### 求各个任务的损失值

In [6]:
def senti_loss( xst, yst, lst, 
                bert, transformer, task_emb,
                senti_classifier, senti_loss_fn,
                cuda=False
               ):
    tensors = words2tensor(bert, xst, cuda) + task_emb
    y_label = torch.tensor(yst.argmax(axis=1)).cuda() if cuda else torch.tensor(yst.argmax(axis=1))
    senti_feature = transformer(tensors.transpose(0, 1)).transpose(0, 1)
    cls_feature = senti_feature.max(axis=1)[0]
    senti_scores = senti_classifier(cls_feature)
    loss = senti_loss_fn(senti_scores, y_label)
    acc, _, _, _, _, _, _ = Count_Accs(y_label, senti_scores.argmax(axis=1))
    return loss, acc

In [7]:
def subj_loss(  xsj, ysj, lsj,
                bert, transformer, task_emb,
                subj_classifier, subj_loss_fn,
                cuda=False
               ):
    tensors = words2tensor(bert, xsj, cuda) + task_emb
    y_label = torch.tensor(ysj.argmax(axis=1)) if not cuda else torch.tensor(ysj.argmax(axis=1)).cuda()
    subj_feature = transformer(tensors.transpose(0, 1)).transpose(0, 1)
    cls_feature = subj_feature.max(axis=1)[0]
    subj_scores = subj_classifier(cls_feature)
    loss = subj_loss_fn(subj_scores, y_label)
    acc, _, _, _, _, _, _ = Count_Accs(y_label, subj_scores.argmax(axis=1))
    return loss, acc

In [8]:
def emotion_loss(   xem, yem, lem,
                    bert, transformer, task_emb,
                    emotion_classifier, emo_loss_fn, 
                    cuda
                ):
    tensors = words2tensor(bert, xem, cuda) + task_emb
    y_label = torch.tensor(yem.argmax(axis=1)) if not cuda else torch.tensor(yem.argmax(axis=1)).cuda()
    emo_feature = transformer(tensors.transpose(0, 1)).transpose(0, 1)
    cls_feature = emo_feature.max(axis=1)[0]
    emo_scores = emotion_classifier(cls_feature)
    loss = emo_loss_fn(emo_scores, y_label)
    acc, _, _, _, _, _, _ = Count_Accs(y_label, emo_scores.argmax(axis=1))
    return loss, acc

### 训练各个任务

In [9]:
def emo_cls_train(emotion_reader, bert, transformer, 
                  task_embedding, emotion_classifier,
                  train_epochs, emo_loss_fn,
                  cuda=False
                 ):
    optim = torch.optim.Adagrad([
                                {'params': bert.parameters(), 'lr':1e-6},
                                {'params': task_embedding.parameters(), 'lr':5e-5},
                                {'params': transformer.parameters(), 'lr': 5e-5},
                                {'params': emotion_classifier.parameters(), 'lr': 5e-5}
                             ]
    )
    losses = np.zeros([10]) 
    accs = np.zeros([10])
    
    task_emb = task_embedding(torch.tensor([2]).cuda()) if cuda else task_embedding(torch.tensor([2]))
    
    optim.zero_grad()
    for epoch in range(train_epochs):
        step = 0
        for x, y, l in emotion_reader.iter():
            emo_loss, emo_acc = emotion_loss(
                                            x, y, l,
                                            bert, transformer, task_emb,
                                            emotion_classifier, emo_loss_fn, 
                                            cuda
                                   )
            emo_loss.backward(retain_graph=True)
            optim.step()
            losses[int(step%10)] = emo_loss.cpu()
            accs[int(step%10)] = emo_acc
            print("step:%d | loss/acc = %.3f/%.3f"%(step, emo_loss, emo_acc))
            if step %10 == 9:
                print('emotion task: %6d: [%5d/%5d], senti_loss/senti_acc = %6.8f/%6.7f ' % ( step,
                                                                                epoch, train_epochs,
                                                                                losses.mean(), accs.mean(),
                                                                            )
                         )       
            step += 1 

In [10]:
def subj_cls_train(subj_reader, bert, transformer, task_embedding,
                   subj_classifier, train_epochs, subj_loss_fn,
                   cuda=False
                  ):
    optim = torch.optim.Adagrad([
                                {'params': bert.parameters(), 'lr':1e-6},
                                {'params': task_embedding.parameters(), 'lr':5e-5},
                                {'params': transformer.parameters(), 'lr': 5e-5},
                                {'params': subj_classifier.parameters(), 'lr': 5e-5}
                             ]
    )
    losses = np.zeros([10]) 
    accs = np.zeros([10])
    task_emb = task_embedding(torch.tensor([1]).cuda()) if cuda else task_embedding(torch.tensor([1]))
    
    optim.zero_grad()
    for epoch in range(train_epochs):
        step = 0
        for x, y, l in subj_reader.iter():
            sj_loss, sj_acc = subj_loss(x, y, l,
                                    bert, transformer, task_emb,
                                    subj_classifier, subj_loss_fn, 
                                    cuda
                                   )
            sj_loss.backward(retain_graph=True)
            optim.step()
            losses[int(step%10)] = sj_loss.cpu()
            accs[int(step%10)] = sj_acc
            print("step:%d | loss/acc = %.3f/%.3f"%(step, sj_loss, sj_acc))
            if step %10 == 9:
                print('subjective task: %6d: [%5d/%5d], senti_loss/senti_acc = %6.8f/%6.7f ' % ( step,
                                                                                epoch, train_epochs,
                                                                                losses.mean(), accs.mean(),
                                                                            )
                         )       
            step += 1 

In [11]:
def senti_cls_train(senti_reader, bert, transformer,
                    task_embedding, senti_classifier,
                    train_epochs, senti_loss_fn,
                    cuda=False
                   ):
    optim = torch.optim.Adagrad([
                                {'params': bert.parameters(), 'lr':1e-6},
                                {'params': task_embedding.parameters(), 'lr':1e-5},
                                {'params': transformer.parameters(), 'lr': 1e-5},
                                {'params': senti_classifier.parameters(), 'lr': 1e-5}
                             ]
    )
    losses = np.zeros([10]) 
    accs = np.zeros([10])
    
    task_emb = task_embedding(torch.tensor([0]).cuda()) if cuda else task_embedding(torch.tensor([0]))
    
    optim.zero_grad()
    for epoch in range(train_epochs):
        step = 0
        for x, y, l in senti_reader.iter():
            st_loss, st_acc = senti_loss(x, y, l, 
                                    bert, transformer, task_emb,
                                    senti_classifier, senti_loss_fn,
                                    cuda
                                   )
            st_loss.backward(retain_graph=True)
            optim.step()
            losses[int(step%10)] = st_loss.cpu()
            accs[int(step%10)] = st_acc
            print("step:%d | loss/acc = %.3f/%.3f"%(step, st_loss, st_acc))
            if step %10 == 9:
                print('sentiment task: %6d: [%5d/%5d], senti_loss/senti_acc = %6.8f/%6.7f ' % ( step,
                                                                                epoch, train_epochs,
                                                                                losses.mean(), accs.mean(),
                                                                            )
                         )       
            step += 1 

### 联合训练函数

In [12]:
rr = np.random.rand(3,10)

In [13]:
rr.mean(axis=1)

array([0.48045042, 0.54320519, 0.42130933])

In [32]:
def JointLearning(senti_reader, subj_reader, emotion_reader, 
                  bert, transformer, task_embedding, 
                  senti_classifier, subj_classifier, emotion_classifier,
                  cuda=False
                 ):
    # stage 1: deploy the trainning on the senti classification task
    senti_weights = torch.tensor(
            WeightsForUmbalanced(
                senti_reader.label
            ),
            dtype=torch.float32
    )
    senti_loss_fn = nn.CrossEntropyLoss(weight=senti_weights.cuda()) if cuda else nn.CrossEntropyLoss(weight=senti_weights)
#     senti_cls_train(
#                     senti_reader, 
#                     bert, 
#                     transformer, 
#                     task_embedding, 
#                     senti_classifier, 
#                     2, 
#                     senti_loss_fn
#                    )
#     senti_reader.reset_batchsize(5)
    #stage 2: deploy the trainning on the subj classification task
    subj_weights = torch.tensor(
            WeightsForUmbalanced(
                subj_reader.label
            ),
            dtype = torch.float32
    )
    subj_loss_fn = nn.CrossEntropyLoss(weight=subj_weights) if not cuda else nn.CrossEntropyLoss(weight=subj_weights.cuda())
#     subj_cls_train(
#                    subj_reader, 
#                    bert, 
#                    transformer, 
#                    task_embedding, 
#                    subj_classifier, 
#                    2, 
#                    subj_loss_fn
#                   )
#     subj_reader.reset_batchsize(5)
    #stage 3: deploy the trainning on the emotion classification task
    emo_weights = torch.tensor(
            WeightsForUmbalanced(
                emotion_reader.label
            ),
            dtype = torch.float32
    )
    emo_loss_fn = nn.CrossEntropyLoss(weight=emo_weights) if not cuda else nn.CrossEntropyLoss(weight=emo_weights.cuda())
#     emo_cls_train(
#         emotion_reader, 
#         bert, 
#         transformer, 
#         task_embedding, 
#         emotion_classifier, 
#         2, 
#         emo_loss_fn
#     )
#     emotion_reader.reset_batchsize(5)    
    
    optim = torch.optim.Adagrad([
                                {'params': bert.parameters(), 'lr':1e-6},
                                {'params': task_embedding.parameters(), 'lr':1e-5},
                                {'params': transformer.parameters(), 'lr': 1e-5},
                                {'params': senti_classifier.parameters(), 'lr': 1e-5},
                                {'params': subj_classifier.parameters(), 'lr':1e-5},
                                {'params': emotion_classifier.parameters(), 'lr':1e-5}
                             ]
    )
    
#     writer = SummaryWriter(log_dir)
    max_epoch = 100
    
    losses = np.zeros([3, 10]) 
    accs = np.zeros([3, 10])
    # [[senti_loss_1, ..., senti_loss_10], [subj_loss_1, ..., subj_loss_10], [emo_loss_1, ..., emo_loss_10]] 
    senti_task_id = torch.tensor([0]) if not cuda else torch.tensor([0]).cuda()
    subj_task_id = torch.tensor([1]) if not cuda else torch.tensor([1]).cuda()
    emo_task_id = torch.tensor([2]) if not cuda else torch.tensor([2]).cuda()
    
    loss_weight = torch.tensor([0.333, 0.333, 0.333]) if not cuda else torch.tensor([0.333, 0.333, 0.333]).cuda()
    
    batchs = min(senti_reader.label.shape[0], subj_reader.label.shape[0], emotion_reader.label.shape[0])
    optim.zero_grad()
    for epoch in range(max_epoch):
        step = 0
        for ((xst, yst, lst), (xsj, ysj, lsj), (xem, yem, lem)) in zip(senti_reader.iter(), subj_reader.iter(), emotion_reader.iter()):
            st_loss, st_acc = senti_loss(xst, yst, lst, 
                                    bert, transformer, task_embedding(senti_task_id),
                                    senti_classifier, senti_loss_fn,
                                    cuda
                                   )
#             st_loss.backward()
            MTL_Loss = st_loss*loss_weight[0]
    
            losses[0][step%10] = st_loss.tolist()
            accs[0][step%10] = st_acc
             
            sj_loss, sj_acc = subj_loss(xsj, ysj, lsj,
                                    bert, transformer, task_embedding(subj_task_id),
                                    subj_classifier, subj_loss_fn,
                                    cuda
                                   )
#             sj_loss.backward()
            MTL_Loss += sj_loss*loss_weight[1]

            losses[1][step%10] = sj_loss.tolist()
            accs[1][step%10] = sj_acc
            
            emo_loss, emo_acc = emotion_loss(xem, yem, lem,
                                    bert, transformer, task_embedding(emo_task_id),
                                    emotion_classifier, emo_loss_fn,
                                    cuda
                                   )
#             emo_loss.backward()
            MTL_Loss += emo_loss*loss_weight[2]
    
            losses[2][step%10] = emo_loss.tolist()
            accs[2][step%10] = emo_acc
            
            MTL_Loss.backward()
            optim.step()
            optim.zero_grad()
            print("%6d|MTL_Loss:%6.8f, senti_loss/senti_acc = %6.8f/%6.7f | subj_loss/subj_acc = %6.8f/%6.7f | emo_loss/emo_acc = %6.8f/%6.7f" % (
                                                                                                step, MTL_Loss,        
                                                                                                losses[0].mean(), accs[0].mean(),
                                                                                                losses[1].mean(), accs[1].mean(),
                                                                                                losses[2].mean(), accs[2].mean()
            )
            )
            if step % 10 == 9:
                print('%6d: [%5d/%5d], MTL_Loss|%6.8f, senti_loss/senti_acc = %6.8f/%6.7f | subj_loss/subj_acc = %6.8f/%6.7f | emo_loss/emo_acc = %6.8f/%6.7f' % (
                                                                                                step,
                                                                                                epoch,max_epoch, MTL_Loss,
                                                                                                losses[0].mean(), accs[0].mean(),
                                                                                                losses[1].mean(), accs[1].mean(),
                                                                                                losses[2].mean(), accs[2].mean()
                                                                                            )
                     )
                loss_weight = torch.tensor(
                                            (1.0/accs.mean(axis=1))/sum(1.0/accs.mean(axis=1)),
                                            dtype=torch.float32
                                )
                print("\n\n loss_weight:", loss_weight)
            step += 1
        joint_model_save_as = '/home/hadoop/ERD/MTLTrain/jointModel_epoch%03d.pkl'% (epoch)
        torch.save(
            {
                "bert":bert.state_dict(),
                "transformer":transformer.state_dict(),
                "task_embedding":task_embedding.state_dict(),
                "senti_classifier": senti_classifier.state_dict(),
                "subj_classifier": subj_classifier.state_dict(),
                "emotion_classifier": emotion_classifier.state_dict()
            },
            rdm_save_as
        )

### 主函数部分

In [15]:
tt = BertTokenizer.from_pretrained("bert-base-uncased", cached_dir = "/home/hadoop/transformer_pretrained_models/bert-base-uncased-pytorch_model.bin")

In [16]:
bb = BertModel.from_pretrained("bert-base-uncased")

In [17]:
task_embedding = nn.Embedding(3, 768)

In [18]:
encoder_layer = nn.TransformerEncoderLayer(768, 8)
transformer_encoder = nn.TransformerEncoder(encoder_layer, 1)

In [19]:
subj_file = "/home/hadoop/rotten_imdb/subj.data"
obj_file = "/home/hadoop/rotten_imdb/obj.data"
tr, dev, te = load_data(subj_file, obj_file)

subj_train_reader = SubjObjReader(tr, 20, tt)
subj_valid_reader = SubjObjReader(dev, 20, tt)
subj_test_reader =  SubjObjReader(te, 20, tt)

In [20]:
subj_cls = nn.Linear(768, 2)

In [21]:
bert = bb.cuda()
transformer = transformer_encoder.cuda()
task_embedding = task_embedding.cuda()
subj_cls = subj_cls.cuda()

### 调试各个任务的部分

#### 调试subjective的任务

In [ ]:
subj_weights = torch.tensor(
            WeightsForUmbalanced(
                subj_train_reader.label
            ),
            dtype = torch.float32
    ).cuda()
subj_loss_fn = nn.CrossEntropyLoss(weight=subj_weights)
subj_cls_train(subj_train_reader, bert, transformer, task_embedding, subj_cls, 1, subj_loss_fn, cuda=True)

> __test code__
``` python
for xsj, ysj, lsj in subj_train_reader.iter():
    loss = subj_loss(  xsj, ysj, lsj,
                bb, transformer_encoder, task_embedding(torch.tensor([1])),
                subj_cls, subj_loss_fn
               )
    break
```
> __train stage__
```python
subj_weights = torch.tensor(
            WeightsForUmbalanced(
                subj_train_reader.label
            ),
            dtype = torch.float32
    ).cuda()
subj_loss_fn = nn.CrossEntropyLoss(weight=subj_weights)
subj_cls_train(subj_train_reader, bert, transformer, task_embedding, subj_cls, 1, subj_loss_fn, cuda=True)
```
_subj task 的训练，在当前的参数配置下，一个ｅｐｏｃｈ可以调到很好的状态_

#### 调试sentiment 的任务

In [22]:
train_file = "/home/hadoop/trainingandtestdata/training.1600000.processed.noemoticon.csv"
test_file = "/home/hadoop/trainingandtestdata/testdata.manual.2009.06.14.csv"
train_set, test_set = tsentiLoader.load_data(train_file, test_file)

/home/hadoop/trainingandtestdata/training.1600000.processed.noemoticon.csv
/home/hadoop/trainingandtestdata/testdata.manual.2009.06.14.csv


In [23]:
senti_train_reader = tsentiLoader.tSentiReader(train_set[:10000], 20, tt)
senti_train_reader.label = np.delete(senti_train_reader.label, 1, axis=2)
# senti_valid_reader = tsentiLoader.tSentiReader(train_set[10000:10100], 20, tt)
# senti_test_reader =  tsentiLoader.tSentiReader(test_set, 20, tt)

In [24]:
senti_cls = nn.Linear(768, 2).cuda()

> __测试sentiment analysis的task__
``` python
senti_weights = torch.tensor(
            WeightsForUmbalanced(
                senti_train_reader.label
            ),
            dtype=torch.float32
    )
senti_loss_fn = nn.CrossEntropyLoss(weight=senti_weights.cuda())
senti_cls_train(senti_train_reader, bert, transformer,
                    task_embedding, senti_cls,
                    2, senti_loss_fn,
                    cuda=True
                   )
``` 
_这个任务至少需要５个ｅｐｏｃｈ才能训练出来个大概的样子_

###### 测试结果


```
step:0 | loss/acc = 0.722/0.500
step:1 | loss/acc = 0.721/0.550
step:2 | loss/acc = 0.735/0.350
step:3 | loss/acc = 0.721/0.350
step:4 | loss/acc = 0.715/0.400
step:5 | loss/acc = 0.748/0.250
step:6 | loss/acc = 0.737/0.400
step:7 | loss/acc = 0.700/0.550
step:8 | loss/acc = 0.734/0.350
step:9 | loss/acc = 0.749/0.450
sentiment task:      9: [    0/    2], senti_loss/senti_acc = 0.72812036/0.4150000 
step:10 | loss/acc = 0.700/0.500
step:11 | loss/acc = 0.690/0.500
step:12 | loss/acc = 0.700/0.500
step:13 | loss/acc = 0.666/0.650
step:14 | loss/acc = 0.663/0.600
step:15 | loss/acc = 0.705/0.400
step:16 | loss/acc = 0.678/0.600
step:17 | loss/acc = 0.673/0.600
step:18 | loss/acc = 0.678/0.550
step:19 | loss/acc = 0.729/0.400
sentiment task:     19: [    0/    2], senti_loss/senti_acc = 0.68820736/0.5300000 
step:20 | loss/acc = 0.693/0.450
step:21 | loss/acc = 0.710/0.300
step:22 | loss/acc = 0.697/0.500
step:23 | loss/acc = 0.659/0.800
step:24 | loss/acc = 0.680/0.550
step:25 | loss/acc = 0.706/0.500
step:26 | loss/acc = 0.714/0.350
step:27 | loss/acc = 0.704/0.600
step:28 | loss/acc = 0.677/0.600
step:29 | loss/acc = 0.667/0.550
sentiment task:     29: [    0/    2], senti_loss/senti_acc = 0.69077948/0.5200000 
step:30 | loss/acc = 0.691/0.550
step:31 | loss/acc = 0.720/0.500
step:32 | loss/acc = 0.683/0.600
step:33 | loss/acc = 0.717/0.500
step:34 | loss/acc = 0.729/0.450
step:35 | loss/acc = 0.651/0.650
step:36 | loss/acc = 0.770/0.400
step:37 | loss/acc = 0.780/0.400
step:38 | loss/acc = 0.614/0.650
step:39 | loss/acc = 0.699/0.500
sentiment task:     39: [    0/    2], senti_loss/senti_acc = 0.70538692/0.5200000 
step:40 | loss/acc = 0.778/0.450
step:41 | loss/acc = 0.767/0.350
step:42 | loss/acc = 0.719/0.550
step:43 | loss/acc = 0.750/0.400
step:44 | loss/acc = 0.670/0.550
step:45 | loss/acc = 0.665/0.500
step:46 | loss/acc = 0.665/0.750
step:47 | loss/acc = 0.682/0.600
step:48 | loss/acc = 0.660/0.550
step:49 | loss/acc = 0.653/0.600
sentiment task:     49: [    0/    2], senti_loss/senti_acc = 0.70080397/0.5300000 
step:50 | loss/acc = 0.698/0.500
step:51 | loss/acc = 0.661/0.700
step:52 | loss/acc = 0.731/0.350
step:53 | loss/acc = 0.697/0.500
step:54 | loss/acc = 0.712/0.450
step:55 | loss/acc = 0.691/0.600
step:56 | loss/acc = 0.686/0.600
step:57 | loss/acc = 0.675/0.600
step:58 | loss/acc = 0.630/0.650
step:59 | loss/acc = 0.710/0.450
sentiment task:     59: [    0/    2], senti_loss/senti_acc = 0.68888318/0.5400000 
step:60 | loss/acc = 0.773/0.350
step:61 | loss/acc = 0.698/0.500
step:62 | loss/acc = 0.612/0.750
step:63 | loss/acc = 0.758/0.250
step:64 | loss/acc = 0.680/0.450
step:65 | loss/acc = 0.727/0.450
step:66 | loss/acc = 0.698/0.500
step:67 | loss/acc = 0.733/0.400
step:68 | loss/acc = 0.660/0.650
step:69 | loss/acc = 0.623/0.700
sentiment task:     69: [    0/    2], senti_loss/senti_acc = 0.69631069/0.5000000 
step:70 | loss/acc = 0.736/0.450
step:71 | loss/acc = 0.664/0.600
step:72 | loss/acc = 0.688/0.550
step:73 | loss/acc = 0.681/0.600
step:74 | loss/acc = 0.672/0.550
step:75 | loss/acc = 0.678/0.650
step:76 | loss/acc = 0.664/0.650
step:77 | loss/acc = 0.658/0.700
step:78 | loss/acc = 0.700/0.500
step:79 | loss/acc = 0.677/0.500
sentiment task:     79: [    0/    2], senti_loss/senti_acc = 0.68184948/0.5750000 
step:80 | loss/acc = 0.743/0.400
step:81 | loss/acc = 0.695/0.650
step:82 | loss/acc = 0.728/0.450
step:83 | loss/acc = 0.678/0.550
step:84 | loss/acc = 0.666/0.650
step:85 | loss/acc = 0.603/0.900
step:86 | loss/acc = 0.652/0.750
step:87 | loss/acc = 0.693/0.450
step:88 | loss/acc = 0.668/0.650
step:89 | loss/acc = 0.685/0.600
sentiment task:     89: [    0/    2], senti_loss/senti_acc = 0.68111634/0.6050000 
step:90 | loss/acc = 0.679/0.650
step:91 | loss/acc = 0.704/0.400
step:92 | loss/acc = 0.662/0.650
step:93 | loss/acc = 0.685/0.450
step:94 | loss/acc = 0.662/0.650
step:95 | loss/acc = 0.618/0.850
step:96 | loss/acc = 0.681/0.550
step:97 | loss/acc = 0.667/0.550
step:98 | loss/acc = 0.669/0.650
step:99 | loss/acc = 0.671/0.750
sentiment task:     99: [    0/    2], senti_loss/senti_acc = 0.66978027/0.6150000 
step:100 | loss/acc = 0.687/0.650
step:101 | loss/acc = 0.663/0.600
step:102 | loss/acc = 0.709/0.500
step:103 | loss/acc = 0.679/0.500
step:104 | loss/acc = 0.663/0.550
step:105 | loss/acc = 0.685/0.550
step:106 | loss/acc = 0.674/0.650
step:107 | loss/acc = 0.670/0.700
step:108 | loss/acc = 0.679/0.550
step:109 | loss/acc = 0.642/0.650
sentiment task:    109: [    0/    2], senti_loss/senti_acc = 0.67511485/0.5900000 
step:110 | loss/acc = 0.656/0.650
step:111 | loss/acc = 0.664/0.700
step:112 | loss/acc = 0.662/0.700
step:113 | loss/acc = 0.719/0.300
step:114 | loss/acc = 0.710/0.400
step:115 | loss/acc = 0.634/0.800
step:116 | loss/acc = 0.636/0.650
step:117 | loss/acc = 0.671/0.600
step:118 | loss/acc = 0.672/0.550
step:119 | loss/acc = 0.662/0.600
sentiment task:    119: [    0/    2], senti_loss/senti_acc = 0.66869312/0.5950000 
step:120 | loss/acc = 0.714/0.500
step:121 | loss/acc = 0.698/0.450
step:122 | loss/acc = 0.701/0.550
step:123 | loss/acc = 0.682/0.600
step:124 | loss/acc = 0.627/0.600
step:125 | loss/acc = 0.649/0.700
step:126 | loss/acc = 0.612/0.800
step:127 | loss/acc = 0.616/0.750
step:128 | loss/acc = 0.689/0.550
step:129 | loss/acc = 0.648/0.700
sentiment task:    129: [    0/    2], senti_loss/senti_acc = 0.66370292/0.6200000 
step:130 | loss/acc = 0.675/0.600
step:131 | loss/acc = 0.609/0.750
step:132 | loss/acc = 0.647/0.750
step:133 | loss/acc = 0.657/0.650
step:134 | loss/acc = 0.679/0.500
step:135 | loss/acc = 0.675/0.600
step:136 | loss/acc = 0.655/0.650
step:137 | loss/acc = 0.696/0.650
step:138 | loss/acc = 0.674/0.600
step:139 | loss/acc = 0.652/0.700
sentiment task:    139: [    0/    2], senti_loss/senti_acc = 0.66198154/0.6450000 
step:140 | loss/acc = 0.675/0.500
step:141 | loss/acc = 0.638/0.700
step:142 | loss/acc = 0.644/0.750
step:143 | loss/acc = 0.660/0.550
step:144 | loss/acc = 0.604/0.700
step:145 | loss/acc = 0.644/0.700
step:146 | loss/acc = 0.741/0.350
step:147 | loss/acc = 0.658/0.500
step:148 | loss/acc = 0.696/0.500
step:149 | loss/acc = 0.684/0.550
sentiment task:    149: [    0/    2], senti_loss/senti_acc = 0.66451840/0.5800000 
step:150 | loss/acc = 0.746/0.350
step:151 | loss/acc = 0.696/0.450
step:152 | loss/acc = 0.704/0.650
step:153 | loss/acc = 0.660/0.550
step:154 | loss/acc = 0.644/0.600
step:155 | loss/acc = 0.674/0.500
step:156 | loss/acc = 0.689/0.600
step:157 | loss/acc = 0.665/0.650
step:158 | loss/acc = 0.732/0.300
step:159 | loss/acc = 0.683/0.550
sentiment task:    159: [    0/    2], senti_loss/senti_acc = 0.68921372/0.5200000 
step:160 | loss/acc = 0.646/0.650
step:161 | loss/acc = 0.616/0.800
step:162 | loss/acc = 0.646/0.650
step:163 | loss/acc = 0.650/0.550
step:164 | loss/acc = 0.688/0.600
step:165 | loss/acc = 0.660/0.700
step:166 | loss/acc = 0.643/0.650
step:167 | loss/acc = 0.687/0.500
step:168 | loss/acc = 0.662/0.550
step:169 | loss/acc = 0.649/0.650
sentiment task:    169: [    0/    2], senti_loss/senti_acc = 0.65483198/0.6300000 
step:170 | loss/acc = 0.669/0.600
step:171 | loss/acc = 0.678/0.550
step:172 | loss/acc = 0.615/0.750
step:173 | loss/acc = 0.658/0.600
step:174 | loss/acc = 0.646/0.750
step:175 | loss/acc = 0.677/0.650
step:176 | loss/acc = 0.683/0.450
step:177 | loss/acc = 0.658/0.650
step:178 | loss/acc = 0.661/0.600
step:179 | loss/acc = 0.725/0.500
sentiment task:    179: [    0/    2], senti_loss/senti_acc = 0.66686422/0.6100000 
step:180 | loss/acc = 0.708/0.500
step:181 | loss/acc = 0.615/0.700
step:182 | loss/acc = 0.644/0.750
step:183 | loss/acc = 0.668/0.450
step:184 | loss/acc = 0.665/0.650
step:185 | loss/acc = 0.622/0.650
step:186 | loss/acc = 0.569/0.950
step:187 | loss/acc = 0.666/0.600
step:188 | loss/acc = 0.637/0.650
step:189 | loss/acc = 0.618/0.750
sentiment task:    189: [    0/    2], senti_loss/senti_acc = 0.64113842/0.6650000 
step:190 | loss/acc = 0.710/0.550
step:191 | loss/acc = 0.672/0.650
step:192 | loss/acc = 0.667/0.600
step:193 | loss/acc = 0.686/0.400
step:194 | loss/acc = 0.584/0.750
step:195 | loss/acc = 0.689/0.500
step:196 | loss/acc = 0.701/0.500
step:197 | loss/acc = 0.645/0.600
step:198 | loss/acc = 0.665/0.600
step:199 | loss/acc = 0.632/0.600
sentiment task:    199: [    0/    2], senti_loss/senti_acc = 0.66506463/0.5750000 
step:200 | loss/acc = 0.707/0.500
step:201 | loss/acc = 0.600/0.750
step:202 | loss/acc = 0.589/0.750
step:203 | loss/acc = 0.668/0.550
step:204 | loss/acc = 0.683/0.600
step:205 | loss/acc = 0.707/0.500
step:206 | loss/acc = 0.689/0.450
step:207 | loss/acc = 0.589/0.800
step:208 | loss/acc = 0.583/0.750
step:209 | loss/acc = 0.703/0.500
sentiment task:    209: [    0/    2], senti_loss/senti_acc = 0.65180305/0.6150000 
step:210 | loss/acc = 0.700/0.400
step:211 | loss/acc = 0.691/0.600
step:212 | loss/acc = 0.661/0.700
step:213 | loss/acc = 0.605/0.750
step:214 | loss/acc = 0.672/0.550
step:215 | loss/acc = 0.583/0.800
step:216 | loss/acc = 0.695/0.450
step:217 | loss/acc = 0.737/0.450
step:218 | loss/acc = 0.673/0.550
step:219 | loss/acc = 0.671/0.550
sentiment task:    219: [    0/    2], senti_loss/senti_acc = 0.66885846/0.5800000 
step:220 | loss/acc = 0.635/0.700
step:221 | loss/acc = 0.693/0.550
step:222 | loss/acc = 0.601/0.750
step:223 | loss/acc = 0.639/0.650
step:224 | loss/acc = 0.582/0.800
step:225 | loss/acc = 0.673/0.600
step:226 | loss/acc = 0.647/0.650
step:227 | loss/acc = 0.665/0.600
step:228 | loss/acc = 0.649/0.700
step:229 | loss/acc = 0.609/0.700
sentiment task:    229: [    0/    2], senti_loss/senti_acc = 0.63940636/0.6700000 
step:230 | loss/acc = 0.674/0.750
step:231 | loss/acc = 0.605/0.900
step:232 | loss/acc = 0.669/0.650
step:233 | loss/acc = 0.661/0.550
step:234 | loss/acc = 0.694/0.500
step:235 | loss/acc = 0.630/0.700
step:236 | loss/acc = 0.651/0.600
step:237 | loss/acc = 0.668/0.550
step:238 | loss/acc = 0.725/0.500
step:239 | loss/acc = 0.691/0.450
sentiment task:    239: [    0/    2], senti_loss/senti_acc = 0.66687196/0.6150000 
step:240 | loss/acc = 0.657/0.600
step:241 | loss/acc = 0.620/0.800
step:242 | loss/acc = 0.684/0.550
step:243 | loss/acc = 0.695/0.600
step:244 | loss/acc = 0.715/0.500
step:245 | loss/acc = 0.629/0.650
step:246 | loss/acc = 0.662/0.550
step:247 | loss/acc = 0.619/0.700
step:248 | loss/acc = 0.702/0.550
step:249 | loss/acc = 0.623/0.700
sentiment task:    249: [    0/    2], senti_loss/senti_acc = 0.66077549/0.6200000 
step:250 | loss/acc = 0.584/0.750
step:251 | loss/acc = 0.612/0.750
step:252 | loss/acc = 0.639/0.600
step:253 | loss/acc = 0.621/0.750
step:254 | loss/acc = 0.700/0.450
step:255 | loss/acc = 0.690/0.500
step:256 | loss/acc = 0.619/0.650
step:257 | loss/acc = 0.620/0.700
step:258 | loss/acc = 0.626/0.700
step:259 | loss/acc = 0.639/0.650
sentiment task:    259: [    0/    2], senti_loss/senti_acc = 0.63497214/0.6500000 
step:260 | loss/acc = 0.649/0.600
step:261 | loss/acc = 0.660/0.700
step:262 | loss/acc = 0.628/0.650
step:263 | loss/acc = 0.579/0.800
step:264 | loss/acc = 0.641/0.650
step:265 | loss/acc = 0.622/0.700
step:266 | loss/acc = 0.653/0.550
step:267 | loss/acc = 0.625/0.700
step:268 | loss/acc = 0.711/0.400
step:269 | loss/acc = 0.664/0.500
sentiment task:    269: [    0/    2], senti_loss/senti_acc = 0.64319305/0.6250000 
step:270 | loss/acc = 0.625/0.700
step:271 | loss/acc = 0.632/0.650
step:272 | loss/acc = 0.679/0.500
step:273 | loss/acc = 0.608/0.750
step:274 | loss/acc = 0.576/0.800
step:275 | loss/acc = 0.668/0.650
step:276 | loss/acc = 0.628/0.550
step:277 | loss/acc = 0.648/0.650
step:278 | loss/acc = 0.588/0.800
step:279 | loss/acc = 0.680/0.500
sentiment task:    279: [    0/    2], senti_loss/senti_acc = 0.63328000/0.6550000 
step:280 | loss/acc = 0.565/0.850
step:281 | loss/acc = 0.584/0.750
step:282 | loss/acc = 0.677/0.550
step:283 | loss/acc = 0.656/0.600
step:284 | loss/acc = 0.648/0.650
step:285 | loss/acc = 0.627/0.700
step:286 | loss/acc = 0.608/0.650
step:287 | loss/acc = 0.674/0.600
step:288 | loss/acc = 0.591/0.650
step:289 | loss/acc = 0.607/0.750
sentiment task:    289: [    0/    2], senti_loss/senti_acc = 0.62377896/0.6750000 
step:290 | loss/acc = 0.639/0.550
step:291 | loss/acc = 0.648/0.550
step:292 | loss/acc = 0.550/0.800
step:293 | loss/acc = 0.610/0.700
step:294 | loss/acc = 0.662/0.650
step:295 | loss/acc = 0.590/0.750
step:296 | loss/acc = 0.652/0.600
step:297 | loss/acc = 0.593/0.700
step:298 | loss/acc = 0.597/0.700
step:299 | loss/acc = 0.615/0.800
sentiment task:    299: [    0/    2], senti_loss/senti_acc = 0.61562613/0.6800000 
step:300 | loss/acc = 0.656/0.650
step:301 | loss/acc = 0.685/0.600
step:302 | loss/acc = 0.621/0.750
step:303 | loss/acc = 0.627/0.650
step:304 | loss/acc = 0.709/0.450
step:305 | loss/acc = 0.677/0.550
step:306 | loss/acc = 0.695/0.500
step:307 | loss/acc = 0.631/0.650
step:308 | loss/acc = 0.664/0.650
step:309 | loss/acc = 0.647/0.600
sentiment task:    309: [    0/    2], senti_loss/senti_acc = 0.66123003/0.6050000 
step:310 | loss/acc = 0.667/0.550
step:311 | loss/acc = 0.678/0.550
step:312 | loss/acc = 0.657/0.600
step:313 | loss/acc = 0.574/0.750
step:314 | loss/acc = 0.577/0.750
step:315 | loss/acc = 0.681/0.550
step:316 | loss/acc = 0.692/0.550
step:317 | loss/acc = 0.651/0.650
step:318 | loss/acc = 0.682/0.500
step:319 | loss/acc = 0.600/0.850
sentiment task:    319: [    0/    2], senti_loss/senti_acc = 0.64592592/0.6300000 
step:320 | loss/acc = 0.604/0.750
step:321 | loss/acc = 0.600/0.750
step:322 | loss/acc = 0.755/0.300
step:323 | loss/acc = 0.584/0.700
step:324 | loss/acc = 0.663/0.600
step:325 | loss/acc = 0.629/0.600
step:326 | loss/acc = 0.603/0.750
step:327 | loss/acc = 0.632/0.600
step:328 | loss/acc = 0.635/0.650
step:329 | loss/acc = 0.652/0.650
sentiment task:    329: [    0/    2], senti_loss/senti_acc = 0.63574392/0.6350000 
step:330 | loss/acc = 0.589/0.850
step:331 | loss/acc = 0.610/0.600
step:332 | loss/acc = 0.681/0.600
step:333 | loss/acc = 0.597/0.800
step:334 | loss/acc = 0.639/0.650
step:335 | loss/acc = 0.636/0.700
step:336 | loss/acc = 0.637/0.700
step:337 | loss/acc = 0.608/0.700
step:338 | loss/acc = 0.655/0.500
step:339 | loss/acc = 0.603/0.700
sentiment task:    339: [    0/    2], senti_loss/senti_acc = 0.62550322/0.6800000 
step:340 | loss/acc = 0.596/0.750
step:341 | loss/acc = 0.636/0.750
step:342 | loss/acc = 0.624/0.650
step:343 | loss/acc = 0.574/0.800
step:344 | loss/acc = 0.638/0.650
step:345 | loss/acc = 0.644/0.650
step:346 | loss/acc = 0.606/0.700
step:347 | loss/acc = 0.695/0.450
step:348 | loss/acc = 0.632/0.700
step:349 | loss/acc = 0.607/0.750
sentiment task:    349: [    0/    2], senti_loss/senti_acc = 0.62520358/0.6850000 
step:350 | loss/acc = 0.628/0.650
step:351 | loss/acc = 0.622/0.700
step:352 | loss/acc = 0.638/0.550
step:353 | loss/acc = 0.617/0.750
step:354 | loss/acc = 0.615/0.700
step:355 | loss/acc = 0.604/0.600
step:356 | loss/acc = 0.577/0.750
step:357 | loss/acc = 0.709/0.450
step:358 | loss/acc = 0.613/0.750
step:359 | loss/acc = 0.672/0.600
sentiment task:    359: [    0/    2], senti_loss/senti_acc = 0.62955916/0.6500000 
step:360 | loss/acc = 0.670/0.600
step:361 | loss/acc = 0.696/0.450
step:362 | loss/acc = 0.598/0.750
step:363 | loss/acc = 0.671/0.600
step:364 | loss/acc = 0.595/0.700
step:365 | loss/acc = 0.691/0.500
step:366 | loss/acc = 0.625/0.750
step:367 | loss/acc = 0.665/0.500
step:368 | loss/acc = 0.584/0.700
step:369 | loss/acc = 0.653/0.550
sentiment task:    369: [    0/    2], senti_loss/senti_acc = 0.64464960/0.6100000 
step:370 | loss/acc = 0.661/0.600
step:371 | loss/acc = 0.611/0.650
step:372 | loss/acc = 0.666/0.450
step:373 | loss/acc = 0.631/0.550
step:374 | loss/acc = 0.551/0.850
step:375 | loss/acc = 0.671/0.600
step:376 | loss/acc = 0.652/0.500
step:377 | loss/acc = 0.652/0.650
step:378 | loss/acc = 0.646/0.550
step:379 | loss/acc = 0.620/0.750
sentiment task:    379: [    0/    2], senti_loss/senti_acc = 0.63593937/0.6150000 
step:380 | loss/acc = 0.610/0.850
step:381 | loss/acc = 0.565/0.900
step:382 | loss/acc = 0.544/0.850
step:383 | loss/acc = 0.589/0.750
step:384 | loss/acc = 0.622/0.650
step:385 | loss/acc = 0.637/0.650
step:386 | loss/acc = 0.601/0.750
step:387 | loss/acc = 0.608/0.700
step:388 | loss/acc = 0.626/0.650
step:389 | loss/acc = 0.673/0.600
sentiment task:    389: [    0/    2], senti_loss/senti_acc = 0.60752185/0.7350000 
step:390 | loss/acc = 0.645/0.600
step:391 | loss/acc = 0.549/0.900
step:392 | loss/acc = 0.641/0.650
step:393 | loss/acc = 0.649/0.650
step:394 | loss/acc = 0.599/0.650
step:395 | loss/acc = 0.621/0.700
step:396 | loss/acc = 0.627/0.550
step:397 | loss/acc = 0.660/0.550
step:398 | loss/acc = 0.644/0.650
step:399 | loss/acc = 0.601/0.750
sentiment task:    399: [    0/    2], senti_loss/senti_acc = 0.62363991/0.6650000 
step:400 | loss/acc = 0.645/0.700
step:401 | loss/acc = 0.576/0.750
step:402 | loss/acc = 0.600/0.850
step:403 | loss/acc = 0.587/0.650
step:404 | loss/acc = 0.660/0.650
step:405 | loss/acc = 0.611/0.600
step:406 | loss/acc = 0.577/0.700
step:407 | loss/acc = 0.675/0.450
step:408 | loss/acc = 0.625/0.600
step:409 | loss/acc = 0.672/0.500
sentiment task:    409: [    0/    2], senti_loss/senti_acc = 0.62274832/0.6450000 
step:410 | loss/acc = 0.593/0.800
step:411 | loss/acc = 0.599/0.650
step:412 | loss/acc = 0.616/0.700
step:413 | loss/acc = 0.578/0.800
step:414 | loss/acc = 0.654/0.600
step:415 | loss/acc = 0.678/0.550
step:416 | loss/acc = 0.553/0.800
step:417 | loss/acc = 0.536/0.800
step:418 | loss/acc = 0.577/0.800
step:419 | loss/acc = 0.650/0.700
sentiment task:    419: [    0/    2], senti_loss/senti_acc = 0.60333155/0.7200000 
step:420 | loss/acc = 0.656/0.500
step:421 | loss/acc = 0.691/0.450
step:422 | loss/acc = 0.640/0.650
step:423 | loss/acc = 0.584/0.850
step:424 | loss/acc = 0.634/0.550
step:425 | loss/acc = 0.659/0.650
step:426 | loss/acc = 0.628/0.650
step:427 | loss/acc = 0.630/0.700
step:428 | loss/acc = 0.544/0.750
step:429 | loss/acc = 0.606/0.550
sentiment task:    429: [    0/    2], senti_loss/senti_acc = 0.62732565/0.6300000 
step:430 | loss/acc = 0.567/0.750
step:431 | loss/acc = 0.653/0.550
step:432 | loss/acc = 0.592/0.750
step:433 | loss/acc = 0.678/0.650
step:434 | loss/acc = 0.658/0.700
step:435 | loss/acc = 0.646/0.600
step:436 | loss/acc = 0.622/0.600
step:437 | loss/acc = 0.599/0.600
step:438 | loss/acc = 0.638/0.600
step:439 | loss/acc = 0.650/0.550
sentiment task:    439: [    0/    2], senti_loss/senti_acc = 0.63022733/0.6350000 
step:440 | loss/acc = 0.659/0.550
step:441 | loss/acc = 0.568/0.700
step:442 | loss/acc = 0.548/0.750
step:443 | loss/acc = 0.605/0.650
step:444 | loss/acc = 0.590/0.650
step:445 | loss/acc = 0.633/0.600
step:446 | loss/acc = 0.655/0.550
step:447 | loss/acc = 0.591/0.700
step:448 | loss/acc = 0.648/0.600
step:449 | loss/acc = 0.551/0.800
sentiment task:    449: [    0/    2], senti_loss/senti_acc = 0.60480673/0.6550000 
step:450 | loss/acc = 0.659/0.550
step:451 | loss/acc = 0.626/0.600
step:452 | loss/acc = 0.739/0.500
step:453 | loss/acc = 0.660/0.650
step:454 | loss/acc = 0.594/0.700
step:455 | loss/acc = 0.559/0.800
step:456 | loss/acc = 0.574/0.800
step:457 | loss/acc = 0.556/0.850
```

#### 调试emotion classification　任务

In [25]:
file2 = "/home/hadoop/EmoNet-PyTorch/twitter30.json"

In [26]:
sentences, labels, emotion_set = load_data_from_file(file2)

In [27]:
emoReader = EmotionReader(sentences, labels, 20, tt)

In [28]:
emo_cls = nn.Linear(768, 6).cuda()

In [25]:
emo_weights = torch.tensor(
            WeightsForUmbalanced(
                emoReader.label
            ),
            dtype = torch.float32
    )
emo_loss_fn = nn.CrossEntropyLoss(weight=emo_weights.cuda())

In [26]:
emo_cls_train(
        emoReader, 
        bert, 
        transformer, 
        task_embedding, 
        emo_cls, 
        1, 
        emo_loss_fn, 
        cuda=True
    )

step:0 | loss/acc = 2.288/0.250
step:1 | loss/acc = 2.018/0.150
step:2 | loss/acc = 1.782/0.350
step:3 | loss/acc = 1.888/0.050
step:4 | loss/acc = 1.767/0.100
step:5 | loss/acc = 2.014/0.000
step:6 | loss/acc = 1.778/0.150
step:7 | loss/acc = 2.181/0.000
step:8 | loss/acc = 1.739/0.050
step:9 | loss/acc = 2.091/0.050
emotion task:      9: [    0/    1], senti_loss/senti_acc = 1.95463120/0.1150000 
step:10 | loss/acc = 2.162/0.050
step:11 | loss/acc = 1.703/0.150
step:12 | loss/acc = 2.002/0.050
step:13 | loss/acc = 2.093/0.000
step:14 | loss/acc = 2.017/0.100
step:15 | loss/acc = 1.796/0.100
step:16 | loss/acc = 1.822/0.150
step:17 | loss/acc = 1.681/0.200
step:18 | loss/acc = 1.757/0.100
step:19 | loss/acc = 1.709/0.250
emotion task:     19: [    0/    1], senti_loss/senti_acc = 1.87408050/0.1150000 
step:20 | loss/acc = 1.793/0.050
step:21 | loss/acc = 1.943/0.000
step:22 | loss/acc = 1.869/0.100
step:23 | loss/acc = 1.848/0.100
step:24 | loss/acc = 1.726/0.200
step:25 | loss/acc = 

KeyboardInterrupt: 

In [ ]:
JointLearning(senti_train_reader, subj_train_reader, emoReader, 
                  bert, transformer, task_embedding, 
                  senti_cls, subj_cls, emo_cls,
                  cuda=True
                 )

     0|MTL_Loss:1.10183227, senti_loss/senti_acc = 0.08993977/0.0500000 | subj_loss/subj_acc = 0.07299789/0.0500000 | emo_loss/emo_acc = 0.16794288/0.0200000
     1|MTL_Loss:1.02234149, senti_loss/senti_acc = 0.16989808/0.1050000 | subj_loss/subj_acc = 0.14532826/0.1000000 | emo_loss/emo_acc = 0.32266364/0.0550000
     2|MTL_Loss:1.21160579, senti_loss/senti_acc = 0.25413162/0.1550000 | subj_loss/subj_acc = 0.22356061/0.1200000 | emo_loss/emo_acc = 0.52404332/0.0700000
     3|MTL_Loss:1.01831877, senti_loss/senti_acc = 0.32044030/0.2200000 | subj_loss/subj_acc = 0.29723995/0.1550000 | emo_loss/emo_acc = 0.68985673/0.1150000
     4|MTL_Loss:1.10961294, senti_loss/senti_acc = 0.40935183/0.2650000 | subj_loss/subj_acc = 0.36816586/0.2000000 | emo_loss/emo_acc = 0.86323639/0.1600000
     5|MTL_Loss:1.19264555, senti_loss/senti_acc = 0.48482938/0.3200000 | subj_loss/subj_acc = 0.44032022/0.2400000 | emo_loss/emo_acc = 1.07375630/0.2050000
     6|MTL_Loss:0.97999442, senti_loss/senti_acc = 0

    47|MTL_Loss:1.17365432, senti_loss/senti_acc = 0.70376768/0.4800000 | subj_loss/subj_acc = 0.70536952/0.5150000 | emo_loss/emo_acc = 1.78908548/0.2750000
    48|MTL_Loss:1.15756273, senti_loss/senti_acc = 0.70553722/0.4650000 | subj_loss/subj_acc = 0.70064408/0.5350000 | emo_loss/emo_acc = 1.79878364/0.2600000
    49|MTL_Loss:1.14014649, senti_loss/senti_acc = 0.70688676/0.4500000 | subj_loss/subj_acc = 0.70046728/0.5250000 | emo_loss/emo_acc = 1.78832955/0.2750000
    49: [    0/  100], MTL_Loss|1.14014649, senti_loss/senti_acc = 0.70688676/0.4500000 | subj_loss/subj_acc = 0.70046728/0.5250000 | emo_loss/emo_acc = 1.78832955/0.2750000


 loss_weight: tensor([0.2862, 0.2454, 0.4684])
    50|MTL_Loss:1.21654522, senti_loss/senti_acc = 0.70895953/0.4400000 | subj_loss/subj_acc = 0.69777147/0.5250000 | emo_loss/emo_acc = 1.80415078/0.2600000
    51|MTL_Loss:1.23796928, senti_loss/senti_acc = 0.70713717/0.4400000 | subj_loss/subj_acc = 0.69424139/0.5350000 | emo_loss/emo_acc = 1.802298